In [ ]:
#Loads packages
import numpy as np
import tifffile
import napari
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu,threshold_local
from skimage.morphology import ball
from skimage.transform import resize
from skimage.measure import label,regionprops
from skimage.exposure import rescale_intensity

import open3d as o3d

This notebook is used to extract areas occupied by pSCRs. The algorithm takes bouton and pSCR segments as an input, and outputs .stl file with a contact surface mesh for further processing in Blender.

Test data includes exemplary segmentation of one bouton and one pSCR.

#### Load segmentation file with boutons and pSCRs

In [ ]:
segmentation = tifffile.imread('.\\data\\bouton_and_pscr.tif')

#### Extract voxels that touch the bouton

In [ ]:
bouton_id = 9

def test_func(values):
    current_px = values[3]
    border = 0
    if current_px not in [bouton_id]:
        for i in range(len(values)):
            if values[i] in [bouton_id]:
                border = current_px
    return border

footprint = np.array([[[0,0,0],
                      [0,1,0],
                      [0,0,0]], 
                     [[0,1,0],
                      [1,1,1],
                      [0,1,0]],
                     [[0,0,0],
                      [0,1,0],
                      [0,0,0]]])

contact_surfaces = ndi.generic_filter(segmentation, test_func, footprint=footprint, mode = 'constant')

In [ ]:
#Sets up napari Viewer and displays MFB segment, associated pSCR segment and the extracted boundary voxels
viewer = napari.Viewer()
viewer.add_labels(segmentation.astype('int16'), name='bouton and pSCRs')
viewer.add_labels(contact_surfaces,name='contact surfaces')

#### Generate 3D meshes from voxels' coordinates

In [ ]:
#Gets coordinates of boundary voxels
rp = regionprops(contact_surfaces)

In [ ]:
#Transforms the coordinates into point cloud data format
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(rp[0].coords)
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=3,
                                                          max_nn=8))
pcd.orient_normals_consistent_tangent_plane(10)
pcd.normalize_normals()

In [ ]:
#Generates and visualizes the mesh
radii = [1,1.5]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([pcd, mesh])

In [ ]:
#Saves mesh as .stl file for further processing in Blender
mesh.scale(0.5, center=(0, 0, 0))
o3d.io.write_triangle_mesh(str(rp[0].label)+'.stl',mesh)